In [1]:
!pip install sentence_transformers
!pip install -q pyvi


from sentence_transformers import SentenceTransformer
from sentence_transformers import util
from pyvi.ViTokenizer import tokenize
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm 
import ast
import re
import random

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 84.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.6 MB/s eta 0:00:00


In [2]:
!gdown --id 1_Pq9VvehWr3d5_2wSkjKf_EVZShyUD8q --output checkpoint-best4.zip
!unzip /kaggle/working/checkpoint-best4.zip


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1_Pq9VvehWr3d5_2wSkjKf_EVZShyUD8q
From (redirected): https://drive.google.com/uc?id=1_Pq9VvehWr3d5_2wSkjKf_EVZShyUD8q&confirm=t&uuid=aa70e428-1c6b-44a0-9c1e-9f07e2d29fad
To: /kaggle/working/checkpoint-best4.zip
100%|██████████████████████████████████████| 1.26G/1.26G [00:30<00:00, 41.0MB/s]
Archive:  /kaggle/working/checkpoint-best4.zip
  inflating: checkpoint-24000/config_sentence_transformers.json  
  inflating: checkpoint-24000/config.json  
  inflating: checkpoint-24000/model.safetensors  
  inflating: checkpoint-24000/tokenizer_config.json  
  inflating: checkpoint-24000/special_tokens_map.json  
  inflating: checkpoint-24000/added_tokens.json  
  inflating: checkpoint-24000/vocab.txt  
  inflating: ch

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentenceTransformer("/kaggle/working/checkpoint-24000", trust_remote_code=True)
model = model.to(device)

In [4]:
def process_context(text):
    if text is None or pd.isna(text) or text == '':
        return []
    text = str(text).strip()

    if text.startswith('[') and text.endswith(']'):
        text = text[1:-1]

    text = re.sub(r"(')(?=\n)", r"\1\n", text)

    items = []
    for item in text.split('\n'):
        item = item.strip()
        item = item.strip("'").strip('"')
        if item:
            items.append(item)

    return items


def fix_format(text):
    text = text.strip()

    text = re.sub(r'\[\s+', '[', text)
    text = re.sub(r'\s+\]', ']', text)

    bracket_content = re.search(r'\[(.*?)\]', text)
    if bracket_content:
        content = bracket_content.group(1)
        numbers = re.split(r'\s+', content.strip())
        fixed_content = ', '.join(filter(None, numbers))
        text = f'[{fixed_content}]'
    
    return text

In [5]:
train = pd.read_csv('/kaggle/input/train-csv/train (1).csv', 
                    dtype={'question': str, 'context': str, 'cid': str, 'qid': str})

train['cid'] = train['cid'].apply(lambda x: ast.literal_eval(fix_format(x)))
train['context'] = train['context'].apply(process_context)

In [6]:
train.head()

,question,context,cid,qid
0,Người học ngành quản lý khai thác công trình t...,"[Khả năng học tập, nâng cao trình độ\n- Khối l...",[62492],161615
1,Nội dung lồng ghép vấn đề bình đẳng giới trong...,[Nội dung lồng ghép vấn đề bình đẳng giới tron...,[151154],80037
2,Sản phẩm phần mềm có được hưởng ưu đãi về thời...,"[Điều 20. Ưu đãi về thời gian miễn thuế, giảm ...",[75071],124074
3,Điều kiện để giáo viên trong cơ sở giáo dục mầ...,"[Điều kiện được hưởng\nCán bộ quản lý, giáo vi...",[225897],146841
4,Nguyên tắc áp dụng phụ cấp ưu đãi nghề y tế th...,"[Nguyên tắc áp dụng\n1. Trường hợp công chức, ...",[68365],6176


In [8]:
def load_embeddings_and_cids(embeddings_path, cids_path):
    embeddings = np.load(embeddings_path)
    print("Embeddings shape:", embeddings.shape) 
    cids = np.load(cids_path)
    print("\nCIDs shape:", cids.shape)

    return embeddings, cids
embeddings, cids = load_embeddings_and_cids("/kaggle/input/new-bi-enocde/embeddings_embeddings.npy", "/kaggle/input/new-bi-enocde/embeddings_cids.npy")

Embeddings shape: (261597, 768)

CIDs shape: (261597, 1)


In [9]:
corpus = pd.read_csv('/kaggle/input/pvi-token/corpus_pvi.csv')  
corpus.head()

,text,cid,tokens
0,"Thông tư này hướng dẫn tuần tra, canh gác bảo ...",0,"Thông_tư này hướng_dẫn tuần_tra , canh_gác bảo..."
1,"1. Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân...",1,"1 . Hàng năm trước mùa mưa , lũ , Ủy_ban nhân_..."
2,Tiêu chuẩn của các thành viên thuộc lực lượng ...,2,Tiêu_chuẩn của các thành_viên thuộc lực_lượng ...
3,"Nhiệm vụ của lực lượng tuần tra, canh gác đê1....",3,"Nhiệm_vụ của lực_lượng tuần_tra , canh_gác đê1..."
4,"Phù hiệu của lực lượng tuần tra, canh gác đêPh...",4,"Phù_hiệu của lực_lượng tuần_tra , canh_gác đêP..."


In [ ]:
train['negative_cid'] = None
train['negative_context'] = None
batch_size = 32

# Xử lý theo batch
for start_idx in tqdm(range(0, len(train), batch_size)):
    end_idx = min(start_idx + batch_size, len(train))
    batch = train.iloc[start_idx:end_idx]
    
    # Encode tất cả questions trong batch
    questions = batch['question'].tolist()
    questions = [tokenize(x) for x in questions]
    query_embeddings = model.encode(questions, show_progress_bar = False)
    
    # Semantic search cho cả batch - lấy top 50 kết quả
    batch_hits = util.semantic_search(query_embeddings, embeddings, top_k=50)
    
    # Xử lý từng row trong batch
    for i, (idx, row) in enumerate(batch.iterrows()):
        positive_cids = {x if isinstance(x, (int, str)) else tuple(x) for x in row['cid']}
        
        candidates = []
        for hit in batch_hits[i]:
            cid = cids[hit['corpus_id']]
            cid_key = cid[0]
            if cid_key not in positive_cids:
                candidates.append(cid[0])
        if len(candidates) > 0:
            num_samples = min(10, len(candidates))
            neg_samples = random.sample(candidates, num_samples)
            
            train.at[idx, 'negative_cid'] = neg_samples
            train.at[idx, 'negative_context'] = [corpus.loc[corpus['cid'] == cid]['text'].values[0] for cid in neg_samples]


  7%|▋         | 263/3733 [03:18<44:35,  1.30it/s]

In [29]:
# # {31682, 31677}
# a = [111513, 31669, 128053, 539405, 121023, 2194, 638958, 128899, 571587, 142960, 
#  31681, 635601, 539393, 539392, 572227, 22395, 31671, 31668, 58131, 548235, 
#  173074, 31672, 539403, 539402, 2140, 87218, 2135, 103821, 90528, 31686, 507159, 
#  539389, 539394, 97476, 552436, 31683, 11807, 176189, 197027, 2132, 548226, 
#  100882, 496734, 444657, 260219, 139432, 548231, 628213]
# len(a)

48

In [30]:
# #{31682, 31677}
# b = [111513, 31669, 128053, 539405, 121023, 2194, 638958, 128899, 571587, 142960, 
#  31677, 31681, 635601, 539393, 539392, 572227, 22395, 31671, 31668, 58131, 
#  548235, 173074, 31672, 539403, 539402, 2140, 87218, 31682, 2135, 103821, 90528, 
#  31686, 507159, 539389, 539394, 97476, 552436, 31683, 11807, 176189, 197027, 2132,
#  548226, 100882, 496734, 444657, 260219, 139432, 548231, 628213]
# len(b)

50

In [ ]:
train.to_csv('train_with_negatives_4.csv', index=False)
